In [1]:
from distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=14)
cluster

2023-04-23 10:37:15,888 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ksm0a69p', purging
2023-04-23 10:37:15,889 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-60ka4nv9', purging
2023-04-23 10:37:15,889 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yg9c4re7', purging
2023-04-23 10:37:15,889 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ltizqxse', purging
2023-04-23 10:37:15,889 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-dlijg40i', purging
2023-04-23 10:37:15,890 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-c3xu6s08', purging
2023-04-23 10:37:15,890 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-lv8tszq5', purging

LocalCluster(0c2c2cd7, 'tcp://127.0.0.1:44817', workers=14, threads=28, memory=57.49 GiB)

In [2]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 14
Total threads: 28,Total memory: 57.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44817,Workers: 14
Dashboard: http://127.0.0.1:8787/status,Total threads: 28
Started: Just now,Total memory: 57.49 GiB
Comm: tcp://127.0.0.1:36721,Total threads: 2
Dashboard: http://127.0.0.1:42943/status,Memory: 4.11 GiB
Nanny: tcp://127.0.0.1:44327,


In [3]:
from scipy.integrate import quad, fixed_quad
import numpy as np
import uproot
from math import pi
from IPython.display import display

from input_parameters import *

print(f"R/l0: {R/l0}")
print(f"Lx/l0: {Lx/l0}")

def get_Bx(x, y, z, x0,y0,z0):
    def integrand_x(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.cos(t)/r_3
    a, b = 0, 2*np.pi
    bx = B0*quad(integrand_x, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bx


def get_By(x, y, z, x0,y0,z0):
    def integrand_y(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*z*np.sin(t)/r_3
    a, b = 0, 2*np.pi
    by = B0*quad(integrand_y, a, b, args=(x-x0, y-y0, z-z0))[0]
    return by

def get_Bz(x, y, z, x0,y0,z0):
    def integrand_z(t, x, y, z):
        r_mag = np.sqrt((x-R*np.cos(t))**2 + (y-R*np.sin(t))**2 + z**2)
        r_3 = r_mag**3
        return R*(R - y*np.sin(t) - x*np.cos(t))/r_3
    a, b = 0, 2*np.pi
    bz = B0*quad(integrand_z, a, b, args=(x-x0, y-y0, z-z0))[0]
    return bz

def two_rings_x(x,y,z):
    Bx_ring1 = get_Bx(x,y,z, x0,y0,z0)
    Bx_ring2 = get_Bx(x,y,z, x1,y1,z1)
    return Bx_ring1 + Bx_ring2

def two_rings_y(x,y,z):
    By_ring1 = get_By(x,y,z, x0,y0,z0)
    By_ring2 = get_By(x,y,z, x1,y1,z1)
    return By_ring1 + By_ring2

def two_rings_z(x,y,z):
    Bz_ring1 = get_Bz(x,y,z, x0,y0,z0)
    Bz_ring2 = get_Bz(x,y,z, x1,y1,z1)
    return Bz_ring1 + Bz_ring2


print(f"Lx: {int(Lx)}")
print(f"Nx: {int(Nx)}")

x = np.linspace(0, Lx, int(Nx))
y = np.linspace(0, Ly, int(Ny))
z = np.linspace(0, Lz, int(Nz))

print("Calculating B components.")
Ax = []
Ay = []
Az = []
num = 0
x_index = 0

dh = display(" ",display_id=True)

def int_row_x(i, j, z):
    Cx = []
    for k in z:
            Cx.append(two_rings_x(i,j,k))
    return Cx
def int_row_y(i, j, z):
    Cy = []
    for k in z:
            Cy.append(two_rings_y(i,j,k))
    return Cy
def int_row_z(i, j, z):
    Cz = []
    for k in z:
            Cz.append(two_rings_z(i,j,k))
    return Cz

for i in x:
    Dx = []
    Dy = []
    Dz = []
    num = num + 1
    dh.update(f"{num} out of {len(x)}")
    for j in y:
        Dx.append(client.submit(int_row_x, i,j,z))
        Dy.append(client.submit(int_row_y, i,j,z))
        Dz.append(client.submit(int_row_z, i,j,z))
    Ax.append(Dx)
    Ay.append(Dy)
    Az.append(Dz)   

R/l0: 3.6
Lx/l0: 16.0
Lx: 100
Nx: 64
Calculating B components.


'64 out of 64'

In [4]:
Bx = client.gather(Ax)
By = client.gather(Ay)
Bz = client.gather(Az)

In [5]:
# Bx = np.flip(By, (1))
# Bx = np.transpose(By, axes=(1,0))
# Bx = np.swapaxes(By, 0,1)

In [6]:
with uproot.recreate("B_field_inputs.root") as f:
    f["B_field"] = {"Bx": Bx, "By": By, "Bz": Bz}
    #f["B_field"] = {"Bx": Bx, "Bz": Bz}

In [7]:
np.shape(Bx)
# np.shape(By)
# np.shape(Bz)

(64, 64, 160)